Connect to the database

Let us now load the ipython-sql extension and establish a connection with the database


In [ ]:
!pip install sqlalchemy==1.3.9

!pip install ibm_db_sa

In [ ]:
%load_ext sql

# Enter the connection string for your Db2 on Cloud database instance below

# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL

%sql ibm_db_sa://pxg97707:7kepN6dBWzvCkKCu@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb?security=SSL

Query the database system catalog to retrieve table metadata
You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [ ]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')

Query the database system catalog to retrieve column metadata
The SCHOOLS table contains a large number of columns. How many columns does this table have?

In [ ]:
# type in your query to retrieve the number of columns in the CHICAGO_CRIME_DATA table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'CHICAGO_CRIME_DATA'

Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [ ]:
# type in your query to retrieve all column names in the CHICAGO_CRIME_DATA table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'CHICAGO_CRIME_DATA'

Problem 1: Find the total number of crimes recorded in the CRIME table.

In [ ]:
%sql select distinct(count(*)) from CHICAGO_CRIME_DATA;

Problem 2: List community areas with per capita income less than 11000.

In [ ]:
%sql select community_area_name, per_capita_income from CENSUS_DATA where per_capita_income < 11000;

Problem 3: List all case numbers for crimes involving minors?

In [ ]:
%sql select case_number from CHICAGO_CRIME_DATA where description like  '%MINOR%';

Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [ ]:
%sql select * from CHICAGO_CRIME_DATA where primary_type='KIDNAPPING' and description like '%CHILD%';

Problem 5: What kind of crimes were recorded at schools?

In [ ]:
%sql select name_of_school, primary_type from CHICAGO_CRIME_DATA c, CHICAGO_PUBLIC_SCHOOLS s\
where c.community_area_number = s.community_area_number limit 5;

Problem 6: List the average safety score for all types of schools.

In [ ]:
%sql select "Elementary, Middle, or High School", avg(safety_score) as average_safety_score from CHICAGO_PUBLIC_SCHOOLS group by "Elementary, Middle, or High School";

Problem 7: List 5 community areas with highest % of households below poverty line.

In [ ]:
%sql select community_area_name, percent_households_below_poverty from CENSUS_DATA\
order by percent_households_below_poverty desc nulls last limit 5;

Problem 8: Which community area(number) is most crime prone?

In [ ]:
%sql select community_area_number, count(*) as Number_of_Crime from CHICAGO_CRIME_DATA group by community_area_number\
order by number_of_crime desc limit 1;

Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [ ]:
%sql select community_area_name from CENSUS_DATA\
where hardship_index = (select max(hardship_index) from CENSUS_DATA)

Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [ ]:
%sql create view num_crime as\
select community_area_number, count(*) as number_of_crime\
from CHICAGO_CRIME_DATA group by community_area_number;
#num_crime = %sql select community_area_number, count(*) as Number_of_Crime from crime group by community_area_number
%sql select * from num_crime limit 5;

In [ ]:
%sql select community_area_name from CENSUS_DATA
    where community_area_number = (select community_area_number from (select community_area_number, count(select community_area_number) as occur
    from chicago_crime_data
    group by community_area_number   
    order by occur desc
    lim 1))                               
                                   